# Wave with periodic bc with the scheme from Palma

We solve the wave equation with Dirichlet boundary conditions (homogeneous) up to $T = 2L$ where $L$ is the space size of the integrating region. The data is reflection symmetric, $u_0(x) = u_0(L-x)$, $\partial_t u(x,t=0) = 0$. This way at the last point the solution repeats itself and we can measure the error very well.

In [ ]:
import Pkg
#Pkg.add("Zygote")
#Pkg.instantiate()
#Pkg.add("StatsBase")
#Pkg.add("UnPack")
#Pkg.add("Revise")
using ComponentArrays
using Distributions
#using WGLMakie
using Lux
using LuxCUDA
using OptimizationOptimJL
using Random
using UnPack
using Zygote
#using CairoMakie  # Backend que funciona en notebooks
using StatsBase
using Revise
using Plots
using JLD2
#import NaNMath

## Status:

Working well in both GPU and CPU without adaptivity. Using both initial data for u and its derivative.

I put some functions here.

In [ ]:
includet("../../neural_tools.jl")

In [ ]:
# -------------------------------------------------------------------
# Configuración
# -------------------------------------------------------------------
config = Dict(
    :N_input => 2,          # [x; t]
    :N_neurons => 40,
    :N_layers => 3,
    :N_output => 1, 
    :N_points => 5_000,     # puntos de colisión (x,t)
    :N_points_bound => 0, # puntos de frontera
    :N_points_0 => 0,    # puntos de condición inicial
    :xmin => 0.0,
    :xmax => 1.0,           # = L dominio espacial
    :tmin => 0.0,           # t_min
    :tmax => 2.0,           # t_max
    :optimizer => BFGS(),
    #:optimizer => SSBroyden(),
    :maxiters => 3_000,
    :N_rounds => 20,    # 5           # nº de rondas RAD
    :iters_per_round => 500,       # iteraciones BFGS por ronda
    :k1 => 1.0, 
    :k2 => 1.0,        # hiperparámetros RAD
    :N_test => 3_000,             # candidatos por ronda, mayor que N_points
    #:method => :adaptive,
    :method => :direct,
    # for the initial data
    :A => 1.0,
    :B => 0.0,
    :x0 => 2.0,
    :x1 => 3.0,
    :p => 2,
    :c => 1.0
)

**Note 1:** I attempted to put a bump function of compact support (with some if's) and could not make it run. It seems there are problems with the AD scheme to handle if's. So I put a simpler function.

**Note 2:** As it is, with adaptive it takes 12 hours (CPU) to run... And the solution is wrong...

In [ ]:
# -------------------------------------------------------------------
# Second order wave equation (onda 1D): u_tt - c^2 u_xx = 0
# -------------------------------------------------------------------

function wave_equation(∂2u_∂x2, ∂2u_∂t2, config) 
    @unpack c
    ∂2u_∂t2 .- (c^2) .* ∂2u_∂x2
end

```
Residual at collocation points
``` 

function residual_at_points_Dirichlet(input, NN, Θ, st)
    x, t = input[1:1, :], input[2:2, :]
    _, u_xx, u_tt = calculate_derivatives_Dirichlet(x, t, NN, Θ, st)
    res = wave_equation(u_xx, u_tt, config[:c])
    return vec(abs.(res |> cpu_device()))  # magnitud del residuo en CPU
end


# -------------------------------------------------------------------
# Loss function with Dirichlet boundary conditions hard enforced.
# -------------------------------------------------------------------

function loss_function(input, NN, Θ, st)
    res = residual_at_points_Dirichlet(input[1], NN, Θ, st)
    #return NaNMath.log10(sum(abs2, res) / length(res))
    return sum(abs2, res) / length(res)
end

#=
function loss_function(input, NN, Θ, st)
    x, t = input[1:1, :], input[2:2, :]
    _, u_xx, u_tt = calculate_derivatives_Dirichlet(x, t, NN, Θ, st)
    res = wave_equation(u_xx, u_tt, config[:c])
    #return NaNMath.log10(sum(abs2, res) / length(res))
    return log10(sum(abs2, res) / length(res))
end
=#

# -------------------------------------------------------------------
# Callback
# -------------------------------------------------------------------
function callback(p, l, losses)
    println("Current loss: ", l)
    push!(losses, log10(l))
    return false
end

Initial data:

In [ ]:
#This way it does not work on GPUS, so we define the function inside the other function
#u0(x) = config[:A]*(x .- xmin).^4 .* (x .- xmax).^4 ./ ((xmax - xmin)/2)^8 # Initial condition
#u1(x) = config[:B]*(x .- xmin).^4 .* (x .- xmax).^4 ./ ((xmax - xmin)/2)^8 # Initial condition for the time derivative

@unpack xmin, xmax, A, B, p = config
#u0(x) = A * (x .- xmin).^4 .* (x .- xmax).^4 ./ ((xmax - xmin)/2)^8
#u1(x) = B * 4.0 .*(x .- xmin).^3 .* (x .- xmax).^3 ./ ((xmax - xmin)/2)^8 .* (2x .- (xmax - xmin))

u0(x) = A*sin.(2π*x*p/(xmax - xmin))
u1(x) = 0.0.*x

### Results:

1. With the p=4 polynomial data works OK and the error is of low frequency.
2. With the p=4 sin data it does not work. Start decreasing the loss function but at some point there is a problem with the optimizer and it does not improve further, on the contrary the loss stabilizes in a larger value.

In [ ]:
# -------------------------------------------------------------------
# Entrenamiento
# -------------------------------------------------------------------
losses = []
NN, Θ, st = create_neural_network(config)
input = generate_input_x_t(config)
input0 = generate_input0_x(config)
input_bound = generate_input_boundary_x(config)
input_total = [input, input0, input_bound]

@show typeof(input_total) size(input_total)
@show get_parameter_count(config)

#input_total[1] === input

In [ ]:
loss_function(input_total, NN, Θ, st)

In [ ]:


#calculate_Dirichlet_f(input[1:1, :], input[2:2, :], NN, Θ, st)
#f, ∂2f_∂x2, ∂2f_∂t2 = calculate_derivatives_Dirichlet(input[1:1, :], input[2:2, :], NN, Θ, st)

In [ ]:
Θ, st, losses = compute_solution_1d(config, input_total, NN, Θ, st, losses)
# Parámetros optimizados a CPU si procede
Θ = optresult.u |> cpu_device()

**Save data:**

In [ ]:
println("Training completed. Saving data")
#@save "wave_Dir_1.jld2" config Θ st losses
#@save "wave_dirichlet_1D_adaptive.jld2" NN Θ st config losses
@save "wave_dirichlet_1D_u0_sin_u1_zero_p4_SSB.jld2" NN Θ st config losses

In [ ]:
plot(losses, label = "Loss", title = "loss vs iterations") #, yscale = :log10)


In [ ]:
losses

config[:tmax]

For the initial data with $u_1(x) = \partial_x u_0(x)$ the solution is:

$$ 
u(x,t) = 
\begin{cases}
& u_0(x+t) \;\;\;\; t \leq x \;\;\text{and} \;\;  t \leq 1 - x \\
& u_0(x+t) - u_0(x-t) \;\;\;\;\; t \geq x \;\; \text{and} \;\; t \leq 1 - x \\
& 0 \;\;\;\;\; t \leq x \;\; \text{and} \;\; t \geq 1 - x \\
& - u_0(x-t) \;\;\;\;\; t \geq x \;\; \text{and} \;\; t \geq 1 - x ;\; \text{and} \;\; t \leq 1 + x \\
\end{cases} 
$$

In [ ]:
#@unpack xmin, xmax, A, B, p = config
t = 1.24
xs = reshape(collect(range(config[:xmin], config[:xmax], length=200)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
#sol_p = [calculate_Dirichlet_f(xs, t_fix, NN, Θ, st) for x in xs]
sol_p = [calculate_Dirichlet_f_wf(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]

plot(xs[1,:],sol_p, label = "approximatio at t=$t", title = "solution")
#lines!(xs[1,:],bump.(xs[1,:], config[:x0], config[:x1], config[:p], config[:A]), label = "initial condition")
plot!(xs[1,:],u0(xs[1,:]), label = "initial condition", linestyle = :dash)  
plot!(xlabel = "x", ylabel = "u(x,t)")   
plot!(xs[1,:], A*(sin.(2π*(xs[1,:].+t)*p/(xmax - xmin)) + sin.(2π*(xs[1,:].-t)*p/(xmax - xmin)))/2, label = "exact solution at t = $t")  # exact solution at t=2.0


In [ ]:
ax = (title = "solution")
t = 1.0
xs = reshape(collect(range(config[:xmin], config[:xmax], length=20000)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
sol_p = [calculate_Dirichlet_f_wf(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]

plot(xs[1,:],sol_p .+ u0.(xs[1,:]), label = "eror at t=1")